# Exploratory Data Analysis of A Year's Pizza Sales
This project analyses a year's worth of sales from anonymised pizza place, including the date and time of each order and the pizzas served, with additional details on the type, size, quantity, price, and ingredients.

In [ ]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# create dataframes from csv files

orders_df = pd.read_csv("orders.csv")
order_details_df = pd.read_csv("order_details.csv")
pizza_types_df = pd.read_csv("pizza_types.csv", encoding ='latin1')
pizzas_df = pd.read_csv("pizzas.csv")

## Ensure data is clean 

### Check orders_df for na and invalid values

In [ ]:

orders_df.info()

In [ ]:
orders_df['date'] = pd.to_datetime(orders_df['date'])
orders_df['hour'] = pd.to_datetime(orders_df['time']).dt.hour
orders_df['day'] = pd.to_datetime(orders_df['date'], format='%Y-%M-%D').dt.day_name()
orders_df['month'] = pd.to_datetime(orders_df['date'], format='%Y-%M-%D').dt.month_name()
orders_df['month_num'] = pd.to_datetime(orders_df['date'], format='%Y-%M-%D').dt.month

In [ ]:
orders_df.head()

In [ ]:
orders_df['date'].value_counts()

In [ ]:
orders_df['time'].value_counts()

### Check order_details_df for null and invalid values

In [ ]:
order_details_df.info()

In [ ]:
order_details_df['order_details_id'].value_counts()

In [ ]:
order_details_df['order_id'].value_counts()

In [ ]:
order_details_df['pizza_id'].value_counts()

In [ ]:
order_details_df['quantity'].value_counts()


### Check pizza_df for null and invalid data

In [ ]:

pizzas_df.info()

In [ ]:
pizzas_df['pizza_id'].value_counts()

In [ ]:
pizzas_df['pizza_type_id'].value_counts()


In [ ]:
pizzas_df['size'].value_counts()


In [ ]:
pizzas_df['price'].value_counts()

### Check pizza_types_df for null and invalid values

In [ ]:
pizza_types_df.info()

## Merge dataframes


In [ ]:
orders_and_details =pd.merge(orders_df, order_details_df, on ='order_id', how ='left')
orders_to_pizza = pd.merge(orders_and_details, pizzas_df, on = 'pizza_id', how ='left')
master_df = pd.merge(orders_to_pizza, pizza_types_df, on = 'pizza_type_id', how ='left')

In [ ]:
master_df

# Analysis

1.	What is the total revenue/sales?

In [ ]:
revenue_per_sale =master_df['quantity']*master_df['price']
total_revenue = revenue_per_sale.sum()
print(f"The total revenue is ${total_revenue}")

2.	Find the total quantity sold

In [ ]:
total_quantity = master_df['quantity'].sum()
print(f"The total number of pizzas sold is {total_quantity}")

3.	Find the total orders.

In [ ]:
print(f"There were {master_df['order_id'].nunique()} orders made in total")

4.	How many pizza types do they sell?

In [ ]:
master_df['pizza_type_id'].nunique()

5.	Find the average price of the pizzas.

In [ ]:
print(f"The average price of pizzas is ${round(master_df['price'].mean(), 2)}")

6.	What are the peak hours of sales?

In [ ]:

sales_per_hour = master_df.groupby('hour')['quantity'].sum().reset_index()
sales_per_hour
plt.figure()
plt.bar(sales_per_hour['hour'],sales_per_hour['quantity'])
plt.plot(sales_per_hour['hour'],sales_per_hour['quantity'], color ='red')
plt.xlabel('Hour of the day')
plt.ylabel('Sum of sales')
plt.title('Sum of Sales by Hour')
plt.show

Insights: There are two daily sales peaks. The major peak at 12 noon and the other peak at 18.00hrs

7.	Find the total sales made on each day of the week. Which day of the week is when sales are made the most?

In [ ]:
sales_per_weekday = master_df.groupby('day')['quantity'].sum().sort_values().reset_index()
plt.axes()
plt.bar(sales_per_weekday['day'],sales_per_weekday['quantity'])
plt.plot(sales_per_weekday['day'],sales_per_weekday['quantity'], color ='red')
plt.xlabel('Day of the week')
plt.ylabel('Sum of sales')
plt.title('Sum of Sales by Weekday')
plt.xticks(rotation=45)
plt.show

Insight: Sales showed an upward trend peaking on Friday. Customers ordered pizzas least on Sundays.

8.	Find the top 5 bestselling pizzas.

In [ ]:
pizza_sales= master_df.groupby('name')['quantity'].sum().reset_index().sort_values(by='quantity', ascending=False).head()
pizza_sales

In [ ]:
plt.axes()
plt.barh(pizza_sales['name'],pizza_sales['quantity'])
plt.title('Top 5 bestselling Pizzas')
plt.ylabel('Pizza Types')
plt.xlabel('Sum of sales')
plt.show()

The chart above shows customers most ordered pizzas for the year

9.	Find the sales made in each month. Any trend noticeable?

In [ ]:
sales_per_month = master_df.groupby(['month_num','month'])['quantity'].sum().reset_index().sort_values('month_num')
plt.axes()
plt.bar(sales_per_month['month'],sales_per_month['quantity'])
plt.plot(sales_per_month['month'],sales_per_month['quantity'], color ='red')
plt.xlabel('Month of the year')
plt.ylabel('Sum of sales')
plt.title('Sum of Sales by Month')
plt.xticks(rotation=45)
plt.show

Insight:

In [ ]:
Sales were stable throughout the year.

10.	Are there pizza types that are not doing well on the menu?

In [ ]:
all_pizza_sales =  master_df.groupby('name')['quantity'].sum().reset_index().sort_values(by='quantity', ascending=True)
poor_pizza_sales = all_pizza_sales[all_pizza_sales['quantity']<1000]
poor_pizza_sales

In [ ]:
plt.axes()
plt.barh(poor_pizza_sales['name'],poor_pizza_sales['quantity'])
plt.title('Poor Performing Pizzas')
plt.ylabel('Pizza Types')
plt.xlabel('Sum of sales')
plt.show()

Insight:
Company policy defines poor sales for any of it's offerings as annual sales below 1000 units. Hence, further investigation is recommended to ascertain causes of low sales for the 10 poorly performing pizzas.